# Light soaking study

### Prends en entrée un fichier Excel .csv
### Fits les spectres de PL
### Recupere une liste des intégrales
### Recupere une liste des bandgaps
### Recupere une liste des FWHM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
%matplotlib inline
from numpy import trapz
from scipy.interpolate import interp1d

In [ ]:
# Chemin du répertoire + nom du fichier
chemin = r'C:\Users\Eva.Uzayambaza\Documents\Eva Documents\Results\PL Results\1 st campaign\Redo 1st campaing20250414\P5-2C CSV.csv'
df = pd.read_csv(chemin, sep=',')
df

In [ ]:
for i in range(0, len(df.columns), 2):
    y  = df.iloc[:, i+1] # intensity
    x = df.iloc[:, i]  # wvl
    y_dark = df.iloc[:, -1]

    # substract background
    corrected = y - y_dark

    plt.plot(x, np.log(corrected), '-', label=df.columns[i])
    plt.xlim(600, 950)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
x = df.iloc[:, 0]
y = df.iloc[:, 1]

interpolation = interp1d(x, y, kind='linear', fill_value='extrapolate')
# Borne d'intégration 
x_inter = np.linspace(650, 900, 1000) #to be adjusted one time
y_inter = interpolation(x_inter)
aire = trapz(y_inter, x_inter)
print("Intégrale de la courbe interpolée :", aire)

plt.plot(x, y, '-', label=df.columns[1])
plt.plot(x_inter, y_inter,'r-', label='Intégrale')
plt.fill_between(x_inter, y_inter, alpha=0.3)
plt.legend()
plt.grid()
plt.show()

In [ ]:
integrale = []
gap = []
fwhm_list = []

for i in range(0, len(df.columns), 2):
    y  = df.iloc[:, i+1] # intensity
    x = df.iloc[:, i]  # wvl
    y_dark = df.iloc[:, -1]
    
    # substract background
    y = y - y_dark
    interpolation = interp1d(x, y, kind='linear', fill_value='extrapolate')
    x_inter = np.linspace(650, 900, 1000)
    y_inter = interpolation(x_inter)
    
    # Integrale
    aire = trapz(y_inter, x_inter)
    print("Intégrale de la courbe interpolée :", aire)
    integrale.append(aire)
    
    # Pic de PL
    val_gap = np.argmax(y_inter)
    x_max = x_inter[val_gap]
    print("Gap de la courbe :", x_max)
    gap.append(x_max)
    
    # FWHM
    y_max = np.max(y_inter)
    half_max = y_max / 2
    indices_above_half_max = np.where(y_inter >= half_max)[0]
    x_left = x_inter[indices_above_half_max[0]]
    x_right = x_inter[indices_above_half_max[-1]]
    fwhm = x_right - x_left
    print("FWHM :", fwhm)
    fwhm_list.append(fwhm)
    
    plt.plot(x, y,'-', label=df.columns[i])
    plt.plot(x_inter, y_inter,'r-', label='Intégrale')
    plt.fill_between(x_inter, y_inter, alpha=0.3)
    plt.xlim(600, 950)
    plt.legend()
    plt.show()
    

In [ ]:
# Conversion en eV
bandgap = []
for i in gap:
    Eg = 1240/i
    bandgap.append(round(Eg, 3))
Eg = bandgap
print(Eg)

In [ ]:
# Sauvegarde dans un dataframe "PL_spectra"
PL_spectra = pd.DataFrame()

for i in range(0, len(df.columns), 2):
    y  = df.iloc[:, i+1] # intensity
    x = df.iloc[:, i]  # wvl
    y_dark = df.iloc[:, -1]
    
    # substract background
    y = y - y_dark
    
    #PL_spectra['Label'] = df.columns[:: 2]
    PL_spectra[df.columns[i]] = x
    PL_spectra[df.columns[i+1]] = y

    # Conversion en csv (dans le dossier ou se situe le code)
    PL_spectra.to_csv('PL_spectra.csv', index=False)

PL_spectra

In [ ]:
# Sauvegarde dans un dataframe "Integrated_PL_gap"
Integrated_PL_peak_fwhm = pd.DataFrame()

Integrated_PL_peak_fwhm['Label'] = df.columns[:: 2]
Integrated_PL_peak_fwhm['Integrated PL Intensity (a.u.)'] = integrale
Integrated_PL_peak_fwhm['PL peak position (eV)'] = Eg
Integrated_PL_peak_fwhm['FWHM (nm)'] = fwhm_list
#Integrated_PL = fichier.dropna()

# Conversion en csv (dans le dossier ou se situe le code)
Integrated_PL_peak_fwhm.to_csv('Integrated_PL_peak_fwhm.csv', index=False)

Integrated_PL_peak_fwhm